# Task Completion Evaluator

## Objective
This sample demonstrates how to use task completion evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing task responses;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **AZURE_AI_PROJECT** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.


The Task Completion evaluator assesses whether an AI agent successfully completes the requested task by examining:
- Whether the task was fully completed
- Quality of task execution
- Appropriateness of the response to the original request

The evaluator uses a binary scoring system (0 or 1):

    - Score 0: The task was not completed or only partially completed
    - Score 1: The task was successfully and fully completed

This evaluation focuses on measuring whether the agent's response indicates successful completion of the user's request, regardless of the specific methods or tools used to achieve the task.

Task Completion requires following input:
- Query - This can be a single query or a list of messages(conversation history with agent). The original task request from the user.
- Response - Response from Agent (or any GenAI App). This can be a single text response or a list of messages generated as part of Agent Response.
- Tool Definitions - (Optional) Tool(s) definition used by Agent to answer the query. Providing tool definitions helps the evaluator better understand the context and capabilities available to the agent.


### Initialize Task Completion Evaluator


In [2]:
import os
from azure.ai.evaluation._evaluators._task_completion import _TaskCompletionEvaluator
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)


task_completion = _TaskCompletionEvaluator(model_config=model_config)

Class _TaskCompletionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Samples

#### Evaluating Simple Task Completion

In [3]:
query = "How is the weather in Seattle?"
response = "The current weather in Seattle is partly cloudy with a temperature of 15°C (59°F). There's a light breeze from the northwest at 8 mph, and the humidity is at 68%. No precipitation is expected for the rest of the day."

# Basic evaluation without tool definitions
result = task_completion(query=query, response=response)
pprint(result)

Conversation history could not be parsed, falling back to original query: How is the weather in Seattle?
Agent response could not be parsed, falling back to original response: The current weather in Seattle is partly cloudy with a temperature of 15°C (59°F). There's a light breeze from the northwest at 8 mph, and the humidity is at 68%. No precipitation is expected for the rest of the day.
Agent response could not be parsed, falling back to original response: The current weather in Seattle is partly cloudy with a temperature of 15°C (59°F). There's a light breeze from the northwest at 8 mph, and the humidity is at 68%. No precipitation is expected for the rest of the day.


{'task_completion': True,
 'task_completion_completion_tokens': 107,
 'task_completion_details': {'completion_gaps': 'None; all relevant weather '
                                                'details were provided.',
                             'delivered_outcome': 'Agent delivered a detailed '
                                                  'weather update with '
                                                  'temperature, cloud cover, '
                                                  'wind, humidity, and '
                                                  'precipitation forecast.',
                             'task_requirements': 'User requested the current '
                                                  'weather conditions in '
                                                  'Seattle.'},
 'task_completion_finish_reason': 'stop',
 'task_completion_model': 'gpt-4.1-2025-04-14',
 'task_completion_prompt_tokens': 1771,
 'task_completion_reason': 'The agent provided a 

#### Task Completion with Tool Context

In [4]:
query = "How is the weather in Seattle?"
response = "I've checked the weather for Seattle and found that it's currently partly cloudy with a temperature of 15°C. There's a light breeze and no rain expected today."

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

result = task_completion(query=query, response=response, tool_definitions=tool_definitions)
pprint(result)

Conversation history could not be parsed, falling back to original query: How is the weather in Seattle?
Agent response could not be parsed, falling back to original response: I've checked the weather for Seattle and found that it's currently partly cloudy with a temperature of 15°C. There's a light breeze and no rain expected today.
Agent response could not be parsed, falling back to original response: I've checked the weather for Seattle and found that it's currently partly cloudy with a temperature of 15°C. There's a light breeze and no rain expected today.


{'task_completion': True,
 'task_completion_completion_tokens': 131,
 'task_completion_details': {'completion_gaps': 'No gaps; all relevant weather '
                                                'details were provided as '
                                                'requested.',
                             'delivered_outcome': 'The agent delivered a '
                                                  'concise weather summary: '
                                                  'partly cloudy, 15°C, light '
                                                  'breeze, and no rain '
                                                  'expected today.',
                             'task_requirements': 'The user asked for the '
                                                  'current weather conditions '
                                                  'in Seattle, expecting a '
                                                  'usable summary of '
                                 

#### Task Completion with Tool Definition as Single Dict

In [5]:
query = "What's the current temperature in Boston?"
response = "The current temperature in Boston is 22°C (72°F) with clear skies. It's a beautiful day with low humidity and a gentle breeze."

tool_definition_dict = {
    "name": "fetch_weather",
    "description": "Fetches the weather information for the specified location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
    },
}

result = task_completion(query=query, response=response, tool_definitions=tool_definition_dict)
pprint(result)

Conversation history could not be parsed, falling back to original query: What's the current temperature in Boston?
Agent response could not be parsed, falling back to original response: The current temperature in Boston is 22°C (72°F) with clear skies. It's a beautiful day with low humidity and a gentle breeze.
Tool definitions could not be parsed, falling back to original definitions: {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}}}


{'task_completion': True,
 'task_completion_completion_tokens': 98,
 'task_completion_details': {'completion_gaps': 'None; all requested '
                                                'information was provided.',
                             'delivered_outcome': 'Agent delivered the current '
                                                  'temperature in both Celsius '
                                                  'and Fahrenheit, plus brief '
                                                  'weather conditions.',
                             'task_requirements': 'User requested the current '
                                                  'temperature in Boston.'},
 'task_completion_finish_reason': 'stop',
 'task_completion_model': 'gpt-4.1-2025-04-14',
 'task_completion_prompt_tokens': 1750,
 'task_completion_reason': 'The agent provided the current temperature in '
                           'Boston, including both Celsius and Fahrenheit, '
                           "w

#### Complex Task with Multiple Steps

In [6]:
query = "Can you send me an email with weather information for Seattle?"
response = [
    {
        "createdAt": "2025-03-26T17:27:35Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I'll get the current weather information for Seattle and then send you an email with the details.",
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:42Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I have successfully sent you an email with the weather information for Seattle. The current weather is partly cloudy with a temperature of 15°C. You should receive the email shortly at your registered email address.",
            }
        ],
    },
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = task_completion(query=query, response=response, tool_definitions=tool_definitions)
pprint(result)

Conversation history could not be parsed, falling back to original query: Can you send me an email with weather information for Seattle?


{'task_completion': True,
 'task_completion_completion_tokens': 133,
 'task_completion_details': {'completion_gaps': 'No gaps if the email was '
                                                'truly sent; otherwise, the '
                                                'task would be incomplete.',
                             'delivered_outcome': 'Agent states an email was '
                                                  "sent with Seattle's weather "
                                                  'details (partly cloudy, '
                                                  "15°C) to the user's "
                                                  'registered address.',
                             'task_requirements': 'Send an email to the user '
                                                  'containing current weather '
                                                  'information for Seattle.'},
 'task_completion_finish_reason': 'stop',
 'task_completion_model': 'gpt-4.1-20

#### Query as Conversation History (List of Messages)
The evaluator also supports query as a list of messages representing conversation history. This helps evaluate task completion in the context of a full conversation.

In [7]:
# Query as conversation history instead of a single string
query_as_conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can fetch weather information and send emails."
    },
    {
        "role": "user", 
        "content": "Hi, I need to plan my day. Can you check the weather in Seattle for me?"
    },
    {
        "role": "user",
        "content": "Also, please send me an email summary of the weather so I can reference it later."
    }
]

response = "I've checked the weather in Seattle for you. It's currently 15°C and partly cloudy with light winds. I've also sent you an email summary with these details so you can reference them throughout your day. The task has been completed successfully."

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = task_completion(query=query_as_conversation, response=response, tool_definitions=tool_definitions)
pprint(result)

Conversation history could not be parsed, falling back to original query: [{'role': 'system', 'content': 'You are a helpful assistant that can fetch weather information and send emails.'}, {'role': 'user', 'content': 'Hi, I need to plan my day. Can you check the weather in Seattle for me?'}, {'role': 'user', 'content': 'Also, please send me an email summary of the weather so I can reference it later.'}]
Agent response could not be parsed, falling back to original response: I've checked the weather in Seattle for you. It's currently 15°C and partly cloudy with light winds. I've also sent you an email summary with these details so you can reference them throughout your day. The task has been completed successfully.
Agent response could not be parsed, falling back to original response: I've checked the weather in Seattle for you. It's currently 15°C and partly cloudy with light winds. I've also sent you an email summary with these details so you can reference them throughout your day. The

{'task_completion': True,
 'task_completion_completion_tokens': 116,
 'task_completion_details': {'completion_gaps': 'None; both weather '
                                                'information and email summary '
                                                'were delivered as requested.',
                             'delivered_outcome': 'Current weather details for '
                                                  'Seattle and confirmation '
                                                  'that an email summary was '
                                                  'sent to the user.',
                             'task_requirements': 'Check Seattle weather and '
                                                  'send an email summary of '
                                                  'the weather for later '
                                                  'reference.'},
 'task_completion_finish_reason': 'stop',
 'task_completion_model': 'gpt-4.1-2025-04-14',
 'ta

#### Example of Incomplete Task

In [8]:
query = "Can you send me an email with weather information for Seattle?"
incomplete_response = "I can see that you want weather information for Seattle. The weather there is usually quite nice this time of year, with temperatures ranging from mild to warm depending on the season."

# This response doesn't complete the email sending task
result = task_completion(query=query, response=incomplete_response, tool_definitions=tool_definitions)
pprint(result)

Conversation history could not be parsed, falling back to original query: Can you send me an email with weather information for Seattle?
Agent response could not be parsed, falling back to original response: I can see that you want weather information for Seattle. The weather there is usually quite nice this time of year, with temperatures ranging from mild to warm depending on the season.
Agent response could not be parsed, falling back to original response: I can see that you want weather information for Seattle. The weather there is usually quite nice this time of year, with temperatures ranging from mild to warm depending on the season.


{'task_completion': False,
 'task_completion_completion_tokens': 128,
 'task_completion_details': {'completion_gaps': 'No email was sent, no '
                                                'specific weather information '
                                                'included, and the deliverable '
                                                'is not actionable or usable '
                                                'as requested.',
                             'delivered_outcome': 'General comment about '
                                                  "Seattle's usual weather, no "
                                                  'email sent, no specific '
                                                  'weather details provided.',
                             'task_requirements': 'Send an email containing '
                                                  'weather information for '
                                                  'Seattle.'},
 'task_completion_fini